# buy-and-hold

buy, then never ever sell, until the end date :)

In [1]:
import pandas as pd
import datetime
import pinkfish as pf

# format price data
pd.options.display.float_format = '{:0.2f}'.format

Some global data

In [2]:
symbol = 'SPY'
capital = 10000
start = datetime.datetime(1900, 1, 1)
end = datetime.datetime.now()

Timeseries

In [3]:
# fetch timeseries, select, finalize
ts = pf.fetch_timeseries(symbol)
ts = pf.select_tradeperiod(ts, start, end, use_adj=True, pad=False)
ts, start = pf.finalize_timeseries(ts, start)

# create tradelog and daily balance objects
tlog = pf.TradeLog(symbol)
dbal = pf.DailyBal()

Algorithm

In [4]:
pf.TradeLog.cash = capital

# loop through timeseries
for i, row in enumerate(ts.itertuples()):

    date = row.Index.to_pydatetime()
    end_flag = pf.is_last_row(ts, i)

    # buy
    if tlog.shares == 0:
        tlog.buy(date, row.close)
    # sell
    elif end_flag:
        tlog.sell(date, row.close)

    # record daily balance
    dbal.append(date, row.high, row.low, row.close)

Retrieve logs

In [5]:
tlog = tlog.get_log()
dbal = dbal.get_log(tlog)

In [6]:
tlog.tail()

,entry_date,entry_price,exit_date,exit_price,pl_points,pl_cash,qty,cumul_total,direction,symbol
0,1993-01-29,26.18,2020-06-22,310.62,284.44,108370.09,381,108370.09,LONG,SPY


Get stats

In [7]:
stats = pf.stats(ts, tlog, dbal, capital)

Summary

In [8]:
pf.summary(stats)

,strategy
annual_return_rate,9.44
max_closed_out_drawdown,-54.22
best_month,21.24
worst_month,-30.73
sharpe_ratio,0.68
sortino_ratio,0.88
monthly_std,4.22
